In [ ]:
from inflation_analysis import grouping

In [ ]:
start_year = 2021
end_year = 2022
years = range(start_year, end_year+1)
base_year = start_year
group_mmb = None 
factor = 1
cex_data_folder = '/Users/roykisluk/Downloads/Consumer_Expenditure_Survey/' 
folder_names_pathname = 'Data_clean/CEX_folder_names.csv' 
prodcode_dict_pathname = 'Data_clean/prodcode_dictionary_c3-c399.csv'

import pandas as pd
import pyreadstat
import numpy as np 


In [ ]:
nationality, observance, income, ses, age, family_size, total_misparmb = grouping(start_year, end_year)

In [ ]:
groups = nationality

In [ ]:
print(groups)

In [ ]:
if base_year is None:   
    base_year = start_year
years = range(start_year, end_year + 1)

groups_mmb = {key: {} for key in groups.keys()}
for key in groups:
    for year in years:
        groups_mmb[key][year] = groups[key][year][['misparmb']]

group_analysis = {}
for key in groups.keys():
    group_number = list(groups.keys()).index(key) + 1
    total_groups = len(groups)
    print(groups_mmb[key])

In [ ]:
group_mmb = groups_mmb['Arab']

In [ ]:

years = range(start_year, end_year + 1)

# Load folder names
folder_names_df = pd.read_csv(folder_names_pathname)

# Functions

# Aggregate total consumption for this group
def total_consumption_value(df): 
    total_consumption = 0.0
    for j in range(0, len(df)):
        total_consumption += df['mehir'][j]
    return total_consumption

# Keep only shared prodcodes for both dataframes (years)
def keep_shared_prodcodes(df1, df2):
    shared_prodcodes = set(df1['prodcode']).intersection(set(df2['prodcode']))
    df1_shared = df1[df1['prodcode'].isin(shared_prodcodes)].reset_index(drop=True)
    df2_shared = df2[df2['prodcode'].isin(shared_prodcodes)].reset_index(drop=True)
    return df1_shared, df2_shared

def weighting(df):
    weights = pd.DataFrame(df['prodcode'].unique(), columns=['prodcode'])
    weights['weight'] = 0.0
    total_consumption = total_consumption_value(df)
    for j in range(0, len(weights)):
        weights.loc[j, 'weight'] = df[df['prodcode'] == weights.loc[j, 'prodcode']]['mehir'].sum() / total_consumption
    return weights

def average_price(df):
    average_prices = pd.DataFrame(df['prodcode'].unique(), columns=['prodcode'])
    average_prices['price'] = 0.0
    for j in range(0, len(average_prices)):
        average_prices.loc[j, 'price'] = (df[df['prodcode'] == average_prices.loc[j, 'prodcode']]['mehir'] / df[df['prodcode'] == average_prices.loc[j, 'prodcode']]['kamut']).mean()
    return average_prices

def Laspeyres(df_base, df_current):
    index_df = pd.DataFrame(df_base['prodcode'].unique(), columns=['prodcode'])
    index_df['index'] = 0.0
    weights = weighting(df_base)
    average_prices_base = average_price(df_base)
    average_prices_current = average_price(df_current)
    index_df = index_df.merge(weights, on='prodcode', how='left')
    index_df = index_df.merge(average_prices_base, on='prodcode', how='left', suffixes=('', '_base'))
    index_df = index_df.merge(average_prices_current, on='prodcode', how='left', suffixes=('_base', '_current'))
    total_index = 0.0
    missing_base_prices = 0
    missing_current_prices = 0
    for j in range(len(index_df)):
        price_current = index_df.loc[j, 'price_current']
        price_base = index_df.loc[j, 'price_base']
        if price_base == 0 or pd.isna(price_base) or np.isinf(price_base):
            index_df.loc[j, 'index'] = factor * 100
            missing_base_prices += 1
            continue
        if price_current == 0 or pd.isna(price_current) or np.isinf(price_current):
            index_df.loc[j, 'index'] = factor * 100
            missing_current_prices += 1
            continue
        index_df.loc[j, 'index'] = (price_current / price_base) * 100
    for j in range(len(index_df)):
        weight = index_df.loc[j, 'weight']
        total_index += weight * index_df.loc[j, 'index']
    return index_df, total_index

def merge_to_secondary(df):
    df['prodcode_secondary'] = df['prodcode'].astype(str).str[:3]
    grouped = df.groupby('prodcode_secondary', group_keys=False).apply(
        lambda x: pd.Series({
            'price_index': np.average(x['index'], weights=x['weight']) if x['weight'].sum() > 0 else np.nan,
            'total_weight': x['weight'].sum()
        }),
        include_groups=False 
    ).reset_index()
    grouped.rename(columns={'prodcode_secondary': 'prodcode'}, inplace=True)
    grouped.rename(columns={'total_weight': 'weight'}, inplace=True)
    return grouped

def merge_to_primary(df):
    df['prodcode_primary'] = df['prodcode'].astype(str).str[:2]
    grouped = df.groupby('prodcode_primary', group_keys=False).apply(
        lambda x: pd.Series({
            'price_index': np.average(x['price_index'], weights=x['weight']) if x['weight'].sum() > 0 else np.nan,
            'total_weight': x['weight'].sum()
        }),
        include_groups=False
    ).reset_index()
    grouped.rename(columns={'prodcode_primary': 'prodcode'}, inplace=True)
    grouped.rename(columns={'total_weight': 'weight'}, inplace=True)
    return grouped

In [ ]:
# Load survey data for each year

dfs_survey = {}
for year in years:
    subfolder = folder_names_df.loc[folder_names_df['Year'] == year, 'Folder_Name'].values[0]
    data_prices_pathname = f"{cex_data_folder}{subfolder}/{subfolder}datayoman.sas7bdat"
    df, meta = pyreadstat.read_sas7bdat(data_prices_pathname)
    df.columns = df.columns.str.lower()
    dfs_survey[year] = df

In [ ]:
print(dfs_survey[2021]['misparmb'].nunique())

In [ ]:
print(group_mmb)

In [ ]:
# Filter observations for relevant group
if group_mmb is not None:
    for year in years:
        dfs_survey[year] = dfs_survey[year][dfs_survey[year]['misparmb'].isin(group_mmb[year]['misparmb'])]


In [ ]:
print(dfs_survey[2021]['misparmb'].nunique())

In [ ]:
print(dfs_survey[2021]['misparmb'].nunique())
print(dfs_survey)

In [ ]:
print(dfs_survey[2021]['prodcode'].min())
print(dfs_survey[2021]['prodcode'].max())

In [ ]:

# Filter observations with prodcode that starts with 3
for year in years:
    dfs_survey[year] = dfs_survey[year][dfs_survey[year]['prodcode'].astype(str).str.startswith('3')].reset_index(drop=True)

In [ ]:
print(dfs_survey[2021]['prodcode'].min())
print(dfs_survey[2021]['prodcode'].max())

In [ ]:

# Calculate weights and price indexes
yearly_price_index = {}
df_price_index = {}
for year in years:
    df_base, df_current = keep_shared_prodcodes(dfs_survey[base_year], dfs_survey[year])
    df_price_index[year], yearly_price_index[year] = Laspeyres(df_base, df_current)

# Combine all years into a single dataframe
combined_df = pd.concat(df_price_index.values(), keys=df_price_index.keys(), names=['Year', 'Index']).reset_index(level='Index', drop=True).reset_index()
combined_df = combined_df[['Year', 'prodcode', 'index', 'weight']]

# Merge to secondary and primary categories
df_secondary = {}
df_primary = {}
for year in years:
    df_secondary[year] = merge_to_secondary(df_price_index[year])
    df_primary[year] = merge_to_primary(df_secondary[year])

# Combine secondary and primary categories into a single dataframe
combined_secondary_df = pd.concat(df_secondary.values(), keys=df_secondary.keys(), names=['Year', 'Index']).reset_index(level='Index', drop=True).reset_index()
combined_primary_df = pd.concat(df_primary.values(), keys=df_primary.keys(), names=['Year', 'Index']).reset_index(level='Index', drop=True).reset_index()

# Keep only the necessary columns
combined_secondary_df = combined_secondary_df[['Year', 'prodcode', 'price_index', 'weight']]
combined_primary_df = combined_primary_df[['Year', 'prodcode', 'price_index', 'weight']]

# Load prodcode dictionary
prodcode_dict_df = pd.read_csv(prodcode_dict_pathname)

# Remove description column if it already exists
if 'description' in combined_secondary_df.columns:
    combined_secondary_df = combined_secondary_df.drop(columns=['description'])
if 'description' in combined_primary_df.columns:
    combined_primary_df = combined_primary_df.drop(columns=['description'])

# Convert prodcode to string in both dataframes before merging
prodcode_dict_df['prodcode'] = prodcode_dict_df['prodcode'].astype(str)
combined_secondary_df['prodcode'] = combined_secondary_df['prodcode'].astype(str)

# Merge descriptions into combined_secondary_df
combined_secondary_df = combined_secondary_df.merge(prodcode_dict_df, on='prodcode', how='left')

# Merge descriptions into combined_primary_df
combined_primary_df = combined_primary_df.merge(prodcode_dict_df, on='prodcode', how='left')
